In [17]:
import numpy as np
import cv2
import os
import uuid

def get_images_and_videos(IMAGE_PATH, VIDEO_PATH, mood, time=5): #time in minutes
    video_final_path = os.path.join(VIDEO_PATH, mood, str(uuid.uuid1()) + ".avi")
    cap = cv2.VideoCapture(0)  
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 30.0, (640, 480))
    total_frames = 30 * time * 60
    count = 0
    while(True):
        ret, frame = cap.read() 
        if count % 60 == 0:
            image_final_path = os.path.join(IMAGE_PATH, str(uuid.uuid1()) + ".jpg")
            cv2.imwrite(image_final_path, frame)
        count += 1
        print(count)
        out.write(frame) 
        cv2.imshow('Original', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if count == total_frames:
            break
    cap.release()
    out.release() 
    cv2.destroyAllWindows()
    


In [19]:
def main():
    IMAGE_PATH = "raw_data/images/"
    VIDEO_PATH = "raw_data/videos/"

    while True:
        print("#######################################################################")
        print("##                                                                   ##")
        print("##   Welcome to video recording session for Mood Detection Project   ##")
        print("##                                                                   ##")
        print("#######################################################################")
        print()
        print()
        print("Choose an option : ")
        print("1. Active")
        print("2. Drowsy")
        print("3. Exit")
        n = int(input("Choose an option : "))
        if n == 1:
            get_images_and_videos(IMAGE_PATH, VIDEO_PATH, "active", 5)
        elif n == 2:
            get_images_and_videos(IMAGE_PATH, VIDEO_PATH, "drowsy", 5)
        elif n == 3:
            break
        else:
            continue

In [22]:
from bs4 import BeautifulSoup

In [23]:
with open('000001_jpg.rf.fddb09e33a544e332617f8ceb53ee805.xml', 'r') as f:
    data = f.read()

bs_data = BeautifulSoup(data, "xml")

In [32]:
bs_data.find("object").find("name").getText()

'helmet'

In [77]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import shutil

def get_xml_path(image_path):
    image_path = str(image_path)
    return image_path[:-3] + "xml"

def get_class(xml_path):
    try:
        with open(xml_path, 'r') as f:
            data = f.read()
        bs_data = BeautifulSoup(data, "xml")
        return bs_data.find_all("object")[-1].find("name").getText()
    except Exception as e:
        return None

def save_image_and_xml(img_path, xml_path, path):
    try:
        shutil.copy(img_path, path)
        shutil.copy(xml_path, path)
        return True
    except Exception as e:
        return False

def train_test_split_images(src, dest):
    train_path = os.path.join(dest, "train")
    test_path = os.path.join(dest, "test")
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)
    src_path = Path(src)
    image_files = src_path.glob("*.jpg")
    image_df = pd.DataFrame()
    image_df["image_path"] = list(map(lambda x:str(x), image_files))
    image_df["xml_path"] = image_df["image_path"].apply(get_xml_path)
    image_df["class"] = image_df["xml_path"].apply(get_class)
    X = image_df.drop(columns=["class"]).values
    y = image_df["class"].values
    for train_ix, test_ix in StratifiedShuffleSplit(n_splits=1, test_size=0.2).split(X, y):
        X_train = X[train_ix]
        X_test = X[test_ix]
        y_train = y[train_ix]
        y_test = y[test_ix]
    list(map(lambda x: save_image_and_xml(x[0], x[1], train_path), X_train))
    list(map(lambda x: save_image_and_xml(x[0], x[1], test_path), X_test))
    

In [55]:
src_path = "C:/Users/ASUS/PycharmProjects/ObjectDetection/workspace/training_demo/images/train/"

In [52]:
categories=["helmet", "head", "person"]

In [54]:
dest = "images/"

In [78]:
df = train_test_split_images(src_path, dest)


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001409_jpg.rf.e2250e9b594423fb01a584c5a0ac0c05.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001409_jpg.rf.e2250e9b594423fb01a584c5a0ac0c05.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004878_jpg.rf.d40e1a80f610079e7266d8b8f6816212.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004878_jpg.rf.d40e1a80f610079e7266d8b8f6816212.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003810_jpg.rf.a6edd94873763c453cbeb16e9267b3d2.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003810_jpg.rf.a6edd94873763c453cbeb16e9267b3d2.xml
images/train

_________________________________________________




C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005219_jpg.rf.799e480ebbdf1d015a3a7223c415ddf1.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005219_jpg.rf.799e480ebbdf1d015a3a7223c415ddf1.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003335_jpg.rf.da93103fca240e96a7ebd014f7c0aa19.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003335_jpg.rf.da93103fca240e96a7ebd014f7c0aa19.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003953_jpg.rf.8c9708e1a88d9dd8e54f14154b216484.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003953_jpg.rf.8c9708e1a88d9dd8e54f14154b216484.xml
images/train

_________________________________________________




C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005143_jpg.rf.0398fcb0a5586a7c28baa8df35cb281a.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005143_jpg.rf.0398fcb0a5586a7c28baa8df35cb281a.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002794_jpg.rf.851f781f0f74e805bf08ab6772aae565.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002794_jpg.rf.851f781f0f74e805bf08ab6772aae565.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002909_jpg.rf.ac0506e0f4e04186736387a175892f0b.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002909_jpg.rf.ac0506e0f4e04186736387a175892f0b.xml
images/train

_________________________________________________




C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\000647_jpg.rf.18eadbe64359cd2f5b74fee3894053fc.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\000647_jpg.rf.18eadbe64359cd2f5b74fee3894053fc.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003171_jpg.rf.11190c184104c3e4fa93b499dd6ccb55.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003171_jpg.rf.11190c184104c3e4fa93b499dd6ccb55.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004685_jpg.rf.4a330cdff1ace6eeec5d840cb95cb2a6.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004685_jpg.rf.4a330cdff1ace6eeec5d840cb95cb2a6.xml
images/train

_________________________________________________




C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003981_jpg.rf.1bb1adcffce3311936da196bb1b9df9d.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003981_jpg.rf.1bb1adcffce3311936da196bb1b9df9d.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003162_jpg.rf.dce1bde4741e0901cf4ff37584e05cd9.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003162_jpg.rf.dce1bde4741e0901cf4ff37584e05cd9.xml
images/train

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\000999_jpg.rf.92988eb7abfcd53c105c74ad015bd064.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\000999_jpg.rf.92988eb7abfcd53c105c74ad015bd064.xml
images/train

_________________________________________________



...


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003848_jpg.rf.5e3c226251d8f7f8552d99dbaa5b742a.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003848_jpg.rf.5e3c226251d8f7f8552d99dbaa5b742a.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004176_jpg.rf.a7076dc4e1d7798a8a1861b5c84e005f.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004176_jpg.rf.a7076dc4e1d7798a8a1861b5c84e005f.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003325_jpg.rf.9b890d7834d6e8b7a46cb2cb4aa41bd0.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003325_jpg.rf.9b890d7834d6e8b7a46cb2cb4aa41bd0.xml
images/test

_________________________________________________

C:\


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004339_jpg.rf.a06945c5c3dd604a970f7e9324b6f0bb.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\004339_jpg.rf.a06945c5c3dd604a970f7e9324b6f0bb.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001336_jpg.rf.81a23f91e88d647f27f35c97c3cfd0b2.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001336_jpg.rf.81a23f91e88d647f27f35c97c3cfd0b2.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003424_jpg.rf.da32dd878f408632b12f9569cefe2d1e.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003424_jpg.rf.da32dd878f408632b12f9569cefe2d1e.xml
images/test

_________________________________________________

C:\


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003734_jpg.rf.43ba39f7480c4f69f667fe03b9a92bd6.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003734_jpg.rf.43ba39f7480c4f69f667fe03b9a92bd6.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002427_jpg.rf.853b2ad513eb09283259a5e9e7ed5ba4.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002427_jpg.rf.853b2ad513eb09283259a5e9e7ed5ba4.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005023_jpg.rf.222f513543c2fde7024edfe2c159bc72.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005023_jpg.rf.222f513543c2fde7024edfe2c159bc72.xml
images/test

_________________________________________________

C:\


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003788_jpg.rf.166c984b9a22eadfb71dc734e3f62177.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\003788_jpg.rf.166c984b9a22eadfb71dc734e3f62177.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001812_jpg.rf.ae4436b0255f6dca4073e9361e3b61ed.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001812_jpg.rf.ae4436b0255f6dca4073e9361e3b61ed.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002048_jpg.rf.9abbb8aef8775efba622584013bcf091.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\002048_jpg.rf.9abbb8aef8775efba622584013bcf091.xml
images/test

_________________________________________________

C:\


C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001272_jpg.rf.067e5eb36ee6190c0e9ba4104e46dedf.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\001272_jpg.rf.067e5eb36ee6190c0e9ba4104e46dedf.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005049_jpg.rf.b2357b974ec5e02517b07f48beede090.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005049_jpg.rf.b2357b974ec5e02517b07f48beede090.xml
images/test

_________________________________________________

C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005208_jpg.rf.a0e357f6cb9e89ee2a7ce78a089c5b5c.jpg
C:\Users\ASUS\PycharmProjects\ObjectDetection\workspace\training_demo\images\train\005208_jpg.rf.a0e357f6cb9e89ee2a7ce78a089c5b5c.xml
images/test

_________________________________________________

C:\

In [68]:
df.iloc[0]["image_path"]

'C:\\Users\\ASUS\\PycharmProjects\\ObjectDetection\\workspace\\training_demo\\images\\train\\000001_jpg.rf.fddb09e33a544e332617f8ceb53ee805.jpg'

In [70]:
shutil.move(df.iloc[0]['image_path'], "images/train/")

'images/train/000001_jpg.rf.fddb09e33a544e332617f8ceb53ee805.jpg'